# Prétraitement d'Images avec OpenCV et PIL/Torchvision

Ce notebook présente deux méthodes de prétraitement d'image :

- **Avec OpenCV** : L'image est lue, convertie de BGR à RGB, redimensionnée et normalisée.
- **Avec PIL et Torchvision** : L'image est chargée avec PIL puis transformée via un pipeline de transformations pour obtenir un tenseur normalisé.

Les deux méthodes sont illustrées avec l'affichage du résultat via `matplotlib`.

In [1]:
import cv2
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch
import os

# Pour afficher correctement les images dans le notebook
%matplotlib inline

## 1. Prétraitement avec OpenCV

La fonction `preprocess_image_cv2` effectue les opérations suivantes :

- Charge l'image en couleur (format BGR par défaut avec OpenCV).
- Convertit l'image de BGR en RGB.
- Redimensionne l'image à une taille donnée (par défaut 224x224).
- Normalise l'image pour que ses valeurs soient comprises entre 0 et 1.

Le résultat est un tableau NumPy.

In [ ]:
def preprocess_image_cv2(image_path, size=(224, 224)):
    """
    Charge et pré-traite l'image avec OpenCV.
    L'image est chargée en couleur (BGR), convertie en RGB,
    redimensionnée et normalisée (mise à l'échelle entre 0 et 1).

    Args:
        image_path (str): Chemin vers l'image.
        size (tuple): Taille souhaitée de l'image (largeur, hauteur).
    
    Returns:
        numpy.ndarray: Image prétraitée.
    """
    # Chargement en couleur (BGR)
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"L'image à {image_path} n'a pas pu être chargée.")
    
    # Conversion BGR -> RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Redimensionnement
    img = cv2.resize(img, size)
    
    # Normalisation : mise à l'échelle entre 0 et 1
    img = img.astype('float32') / 255.0
    return img

## 2. Pipeline de Transformation avec Torchvision

Ici, nous définissons un pipeline de transformation pour préparer l'image avec PIL :

- Redimensionnement à 256 pixels.
- Découpage centré à 224 pixels.
- Conversion en tenseur (les valeurs sont mises à l'échelle entre 0 et 1).
- Normalisation à l'aide de moyennes et écarts-types prédéfinis.

Ce pipeline prépare l'image pour l'utilisation avec des modèles de deep learning.

In [ ]:
transform_pipeline = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # Conversion en tensor avec valeurs dans [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## 3. Prétraitement avec PIL et Torchvision

La fonction `preprocess_image_pil` utilise PIL pour charger l'image et applique ensuite le pipeline de transformation défini ci-dessus. 

Le résultat est un tenseur prétraité, adapté aux modèles de deep learning.

In [ ]:
def preprocess_image_pil(image_path):
    """
    Charge et pré-traite l'image en utilisant PIL et le pipeline Torchvision.
    L'image est convertie en format PIL, puis transformée en tensor.

    Args:
        image_path (str): Chemin vers l'image.
    
    Returns:
        torch.Tensor: Image prétraitée sous forme de tensor.
    """
    img = Image.open(image_path).convert('RGB')
    return transform_pipeline(img)

## 4. Exemple d'Utilisation

Dans cette section, nous appliquons les deux méthodes de prétraitement sur une image d'exemple. 

Nous vérifions d'abord que le fichier existe. Ensuite, nous :

- Prétraitons l'image avec OpenCV et affichons le résultat.
- Prétraitons l'image avec PIL & Torchvision, réarrangeons les dimensions pour l'affichage, puis affichons le résultat.

Note : En raison de la normalisation, les valeurs du tenseur peuvent être négatives ou dépasser 1. Pour l'affichage, nous remodulons ces valeurs entre 0 et 1.

In [ ]:
if __name__ == "__main__":
    # Chemin de l'image (utiliser une chaîne brute pour gérer les espaces et les antislashs)
    image_path = r"C:\Users\HP\Desktop\LIPSTIP\Logos_dataset\earlier_005271598.jpg"
    
    # Vérification de l'existence du fichier
    if not os.path.exists(image_path):
        print(f"Le fichier n'existe pas : {image_path}")
    else:
        # Prétraitement avec OpenCV
        try:
            processed_img_cv2 = preprocess_image_cv2(image_path)
            plt.figure(figsize=(6, 6))
            plt.imshow(processed_img_cv2)
            plt.title("Prétraitement avec OpenCV")
            plt.axis('off')
            plt.show()
        except Exception as e:
            print("Erreur lors du prétraitement avec OpenCV :", e)

        # Prétraitement avec PIL et Torchvision
        try:
            processed_img_pil = preprocess_image_pil(image_path)
            # Réarrangement des dimensions pour l'affichage (C, H, W) -> (H, W, C)
            img_np = processed_img_pil.permute(1, 2, 0).numpy()
            # Remise à l'échelle pour l'affichage (optionnelle)
            img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())
            plt.figure(figsize=(6, 6))
            plt.imshow(img_np)
            plt.title("Prétraitement avec PIL & Torchvision")
            plt.axis('off')
            plt.show()
        except Exception as e:
            print("Erreur lors du prétraitement avec PIL & Torchvision :", e)